In [32]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [34]:
class ParentState(TypedDict):

    question: str
    answer_eng: str
    answer_roman: str

In [35]:
parent_llm = ChatOpenAI(model='gpt-4o-mini')
subgraph_llm = ChatOpenAI(model='gpt-4o')

In [36]:
def translate_text(state: ParentState):

    prompt = f"""
        Translate the following text to Roman Urdu.
        Keep it natural and clear. Do not add extra content.

        Text:
        {state["answer_eng"]}
    """.strip()
    
    translated_text = subgraph_llm.invoke(prompt).content

    return {'answer_roman': translated_text}

In [37]:
subgraph_builder = StateGraph(ParentState)

subgraph_builder.add_node('translate_text', translate_text)

subgraph_builder.add_edge(START, 'translate_text')
subgraph_builder.add_edge('translate_text', END)

subgraph = subgraph_builder.compile()

In [38]:
def generate_answer(state: ParentState):

    answer = parent_llm.invoke(f"You are a helpful assistant. Answer clearly.\n\nQuestion: {state['question']}").content
    return {'answer_eng': answer}

In [39]:
parent_builder = StateGraph(ParentState)

parent_builder.add_node("answer", generate_answer)
parent_builder.add_node("translate", subgraph)

parent_builder.add_edge(START, 'answer')
parent_builder.add_edge('answer', 'translate')
parent_builder.add_edge('translate', END)

In [40]:
graph = parent_builder.compile()

In [41]:
final_state = graph.invoke({'question': 'What is quantum physics'})

sections = []
for key, value in final_state.items():
    sections.append(f"### **{key.capitalize()}**\n{value}\n")

output_text = "\n---\n".join(sections)
display(Markdown(output_text))

### **Question**
What is quantum physics

---
### **Answer_eng**
Quantum physics, also known as quantum mechanics, is a fundamental branch of physics that describes the behavior of matter and energy at the smallest scales, typically at the level of atoms and subatomic particles. It differs significantly from classical physics in its principles and predictions.

Key features of quantum physics include:

1. **Wave-Particle Duality**: Particles, such as electrons and photons, exhibit both particle-like and wave-like behaviors. This means they can behave like discrete particles in some experiments and like waves in others.

2. **Quantization**: Energy, momentum, and other quantities are quantized, meaning they can only take on discrete values rather than a continuous range. For example, electrons in an atom can only exist in specific energy levels.

3. **Uncertainty Principle**: Formulated by Werner Heisenberg, this principle states that certain pairs of physical properties (like position and momentum) cannot be simultaneously measured with arbitrary precision. The more accurately one property is known, the less accurately the other can be known.

4. **Entanglement**: Quantum particles can become entangled, meaning the state of one particle is directly related to the state of another, regardless of the distance separating them. Changes to one particle can instantaneously affect the other.

5. **Superposition**: Particles can exist in multiple states or configurations at once, a phenomenon known as superposition. It's only upon measurement that the system 'collapses' into one of the possible states.

Quantum physics is fundamental to many modern technologies, including semiconductors, lasers, and quantum computing, and it plays a crucial role in our understanding of the universe at a fundamental level.

---
### **Answer_roman**
Quantum physics, jisay quantum mechanics bhi kaha jata hai, physics ki ek buniyadi shaakha hai jo mawad aur energy ke ravaiyya ko sabse choti scale par, aam tor par atom aur subatomic zarrat ke level par, bayan karti hai. Ye classical physics se apni usoolon aur peshgoiyon mein khas tor par mukhtalif hai.

Quantum physics ke ahem pehlu shamil hain:

1. **Wave-Particle Duality**: Zarrat, jaise ke electrons aur photons, particle-like aur wave-like ravaiyya pradarshit karte hain. Iska matlab ye hai ke woh kuch tajurbaat mein discrete particles ki tarah behave karte hain aur doosron mein lehron ki tarah.

2. **Quantization**: Energy, momentum aur doosri miktarein quantized hain, yani woh sirf discrete values le sakti hain bajaye ke ek musalsal range. Misal ke tor par, ek atom mein electrons sirf khas energy levels par maujood ho sakte hain.

3. **Uncertainty Principle**: Werner Heisenberg ke banaye hue iss usool kehte hain ke chand fiziki khususiyaat (jaise ke position aur momentum) ko aik sath man chahi daqiqat ke sath maapay nahi ja sakta. Jitni ziada mehkak ek khususiyaat ki roshni mein milti hai, utni hi kami dusri ki daqiqat mein hoti hai.

4. **Entanglement**: Quantum zarrat entangled ho sakti hain, yani ek zarra ki haalat seedha doosray zarra ke haalat se taluq rakhti hai, chahe unke darmiyan kitna hi faasla kyun na ho. Ek zarra mein tabdili foran doosray par asar andaz hoti hai.

5. **Superposition**: Zarrat ek waqt mein multiple states ya configurations mein mojood ho sakti hain, jo ke superposition kehlata hai. Sirf pemaish ke waqt hi system ek mumkin haalat mein collapse hota hai.

Quantum physics bohot si jadid teknolojion, jese ke semiconductors, lasers, aur quantum computing, ke liye buniyadi hai, aur yeh kainaat ke ek buniyadi satah par samajhne mein ahem kirdar ada karti hai.
